# Model Training in SageMaker

In this notebook we will be using Amazon's Built-in Image Classifier to train on our image dataset. Additionally we will be performing hyperparameter tuning to get the best combination of hyperparameters that yield the best results.

- load docker container containing image classifier
- set up training and hyperparameters for image classifier
- configure the hyperparameter tuning job
- setting up the data channels
- run the hyperparameter tuning job

In [2]:
import sagemaker
import boto3
import os

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
bucket = 'not-hot-dog'

os.environ['DEFAULT_s3_BUCKET'] = bucket

In [3]:
print(role)
print(region)

arn:aws:iam::003294323742:role/service-role/AmazonSageMaker-ExecutionRole-20230115T105791
us-east-2


## Load Docker Container

SageMaker provides docker containers for different built-in algorithms. We will be loading the docker container containing the image classification model.

In [4]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = sagemaker.image_uris.retrieve(
    region = region,
    framework = 'image-classification'
)

In [5]:
# Setting up where out trained machine learning model will be stored.
s3_output_location = 's3://{}/models/image_model'.format(bucket)

## Setting up Parameters and Hyperparameters

### Training Parameters

In [6]:
#Setting up training parameters
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size = 50
max_run = 360000
input_mode = 'File'

In [7]:
image_classifier = sagemaker.estimator.Estimator(
    container,
    role = role,
    instance_count = instance_count,
    instance_type = instance_type,
    volume_size = volume_size,
    max_run = max_run,
    input_mode = input_mode,
    output_path = s3_output_location,
    sagemaker_session = sagemaker_session
)

### Hyperparameters

In [9]:
use_pretrained_model = 1
image_shape = '3,224,224'
num_classes = 2
num_training_samples = 799
epochs = 15
augmentation_type = 'crop_color_transform'
early_stopping_min_epochs = 8
early_stopping_patience = 5
early_stopping_tolerance = 0.0
lr_scheduler_factor = 0.1
lr_scheduler_step = '8,10,12'

image_classifier.set_hyperparameters(
    use_pretrained_model = use_pretrained_model,
    image_shape = image_shape,
    num_classes = num_classes,
    num_training_samples = num_training_samples,
    epochs = epochs,
    augmentation_type = augmentation_type,
    early_stopping_patience = early_stopping_patience,
    early_stopping_tolerance = early_stopping_tolerance,
    lr_scheduler_factor = lr_scheduler_factor,
    lr_scheduler_step = lr_scheduler_step
)

## Configure Hyperparameter Tuning Job

In [11]:
# Configure hyperparameter tuning
import time
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.01, 0.1),
    'mini_batch_size': CategoricalParameter([8, 16, 32]),
    'optimizer': CategoricalParameter(['sgd', 'adam'])
}

In [12]:
tuner = HyperparameterTuner(
    estimator = image_classifier,
    objective_metric_name = 'validation:accuracy',
    objective_type = 'Maximize',
    max_jobs = 5,
    max_parallel_jobs = 1,
    hyperparameter_ranges = hyperparameter_ranges
)

## Setting up Data Channels

In [19]:
s3_train_uri = 's3://{}/train/'.format(bucket)
s3_validation_uri = 's3://{}/validation/'.format(bucket)
s3_train_lst_uri = 's3://{}/train.lst'.format(bucket)
s3_validation_lst_uri = 's3://{}/validation.lst'.format(bucket)

In [21]:
from sagemaker.session import TrainingInput

# creating training data inputs
s3_input_train_data = sagemaker.inputs.TrainingInput(
    s3_data = s3_train_uri,
    content_type = 'application/x-image'
)

# creating validation data inputs
s3_input_validation_data = sagemaker.inputs.TrainingInput(
    s3_data = s3_validation_uri,
    content_type = 'application/x-image'
)

# creating train lst input
s3_input_train_lst = sagemaker.inputs.TrainingInput(
    s3_data = s3_train_lst_uri,
    content_type = 'application/x-image'
)

# creating validation lst input
s3_input_validation_lst = sagemaker.inputs.TrainingInput(
    s3_data = s3_validation_lst_uri,
    content_type = 'application/x-image'
)

# Create data channel
data_channels = {
    'train': s3_input_train_data,
    'validation':  s3_input_validation_data,
    'train_lst': s3_input_train_lst,
    'validation_lst': s3_input_validation_lst
}

## Run Hyperparameter Tuning Job

In [22]:
tuner.fit(
    inputs = data_channels,
    include_cls_metadata = False,
    wait = False
)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


In [23]:
tuning_job_name = tuner.latest_tuning_job.job_name
print(tuning_job_name)

image-classification-230118-0234


In [24]:
%%time 

tuner.wait()

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................